In [3]:
from transformers import pipeline

In [7]:
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [18]:
def analyse_sentiment(text: str) -> dict:
    result = sentiment_pipeline(text)

    # assign sentiment and confidence
    sentiment = result[0].get('label')
    confidence = result[0].get('score')

    return {
        "sentiment": sentiment,
        "confidence": confidence
    }


In [19]:
analyse_sentiment("This is amazing!")

{'sentiment': 'positive', 'confidence': 0.9747719764709473}